In [64]:
import numpy as np
import tensorflow as tf

In this notebook, there are three evaluation metrics used in ranking or recommendation system which are
1. Mean Average Precision (MAP)
2. Precision@k
3. Normalized Discounted Cumulative Gain  (NDCG)

1. Mean Average Precision (MAP)
According to the link https://www.youtube.com/watch?v=pM6DJ0ZZee0, Mean Average Precision (MAP) is the standard single-number measure for comparing search algorithms. Average precision (AP) is the average of (un-interpolated) precision values at all ranks where relevant documents are found. 

However, we need to compute AP first then average them to get MAP.

The function apk is modified from https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
To get the result the same as AP, we have to define k= length of predicted list. 

For example, given actual=[1], predicted = [1,1,3,4,1], and k=5. The apk calculates the precision:
i=1 p = 1/1
i=2 p = 2/2
i=3 rel = 0
i=4 rel = 0
i=5 p = 3/5
So, apk@5 = (1 + 1 + 0.6) / 3 =  0.8666666666666667;

In [65]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]
    else:
        k=len(predicted)
        
    predicted=np.array(predicted).astype(float).reshape(1,k)
    y_true= np.array(actual).astype(float).reshape(len(actual),1)
    onesmatrix=np.ones_like(predicted)
    compare_matrix=np.dot(y_true,onesmatrix)
    cp=1.0*(compare_matrix==predicted)

    length=np.arange(1,k+1,1)

    if not actual:
        return 0.0

    return np.sum(np.cumsum(cp)*cp/length)/np.sum(cp)
  

In [66]:
actual = [1]
k=5
predicted = [1,1,3,4,1]
print('Answer=',actual,'predicted=',predicted)
print('AP@{} ='.format(k),apk(actual,predicted,k) )

('Answer=', [1], 'predicted=', [1, 1, 3, 4, 1])
('AP@5 =', 0.8666666666666667)


The function mapk computes the mean average precision at k among lists of items.
As mentioned before, by defining k=length of a predicted list, we get the MAP.

For example, given actual=[[1],[1],[1],[1],[1]] and 
predicted=[[1,1,3,4,1],[2,1,3,4,5],[3,2,1,4,5],[4,2,3,1,5],[4,2,3,5,1]]
For each input list:
i=1  apk= 0.8666666666666667
i=2  apk= 0.5
i=3  apk= 0.3333333333333333
i=4  apk= 0.25
i=5  apk= 0.2
So, mapk@5 = (0.8666666666666667 + 0.5 + 0.3333333333333 +0.25 +0.2) / 5 =  0.43;

In [67]:
def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
            
            https://www.kaggle.com/wendykan/map-k-demo
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [68]:
actual=[[1],[1],[1],[1],[1]]
predicted=[[1,1,3,4,1],[2,1,3,4,5],[3,2,1,4,5],[4,2,3,1,5],[4,2,3,5,1]]
print('MAP@5 =',mapk(actual, predicted, k=5))

('MAP@5 =', 0.43)


2. Precision at K
According to Evaluation Metrics for Model Selection in Ranking slide, Precision at k’ is the proportion of recommended items in the top-k set that are relevant.
Suppose that my precision at 10 in a top-10 recommendation items is 80% (0.8). This means that 8 out of 10 items that recommended are relevant to the user.
Precision@k = (no. relevant items of k-recommended) / k

For example, given actual=[1], predicted = [1,1,3,4,1], and k=5. The pk calculates the precision at k:
So, pk@5 = (1 + 1 + 0 +0 + 1) / 5 =  0.6;

In [69]:
def pk(actual, predicted, k=10):
    """
    Computes the precision at k.
    This function computes the prscision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    y_true= np.array(actual)
    onesmatrix=np.ones_like((actual))
    compare_matrix=np.dot(actual,onesmatrix)

    if not actual:
        return 0.0

    return np.mean(compare_matrix == predicted)

In [70]:
actual = [1]
k=5
predicted = [1,1,3,4,1]
print('Answer=',actual,'predicted=',predicted)
print('PK@{} =',format(k), pk(actual,predicted,k) )

('Answer=', [1], 'predicted=', [1, 1, 3, 4, 1])
('PK@{} =', '5', 0.6)


In [71]:
actual = [1]
k=5
predicted = [1,1,3,4,1]
print('Answer=',actual,'predicted=',predicted)
print('AP@{} ='.format(k),apk(actual,predicted,k) )
print('PK@{} ='.format(k), pk(actual,predicted,k) )

predicted = [2,1,3,4,5]
print('Answer=',actual,'predicted=',predicted)
print('AP@{} ='.format(k),apk(actual,predicted,k) )
print('PK@{} ='.format(k), pk(actual,predicted,k) )

predicted = [3,2,1,4,5]
print('Answer=',actual,'predicted=',predicted)
print('AP@{} ='.format(k),apk(actual,predicted,k) )
print('PK@{} ='.format(k), pk(actual,predicted,k) )

predicted = [4,2,3,1,5]
print('Answer=',actual,'predicted=',predicted)
print('AP@{} ='.format(k),apk(actual,predicted,k) )
print('PK@{} ='.format(k), pk(actual,predicted,k) )

predicted = [4,2,3,5,1]
print('Answer=',actual,'predicted=',predicted)
print('AP@{} ='.format(k),apk(actual,predicted,k) )
print('PK@{} ='.format(k), pk(actual,predicted,k) )

('Answer=', [1], 'predicted=', [1, 1, 3, 4, 1])
('AP@5 =', 0.8666666666666667)
('PK@5 =', 0.6)
('Answer=', [1], 'predicted=', [2, 1, 3, 4, 5])
('AP@5 =', 0.5)
('PK@5 =', 0.2)
('Answer=', [1], 'predicted=', [3, 2, 1, 4, 5])
('AP@5 =', 0.3333333333333333)
('PK@5 =', 0.2)
('Answer=', [1], 'predicted=', [4, 2, 3, 1, 5])
('AP@5 =', 0.25)
('PK@5 =', 0.2)
('Answer=', [1], 'predicted=', [4, 2, 3, 5, 1])
('AP@5 =', 0.2)
('PK@5 =', 0.2)


In [72]:
def mpk(actual, predicted, k=10):
    """
    Computes the mean precision at k.
    This function computes the mean precision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns: The average precision at k over the input lists
            
            https://www.kaggle.com/wendykan/map-k-demo
    """
    return np.mean([pk(a,p,k) for a,p in zip(actual, predicted)])

In [73]:
actual=[[1],[1],[1],[1],[1]]
predicted=[[1,1,3,4,1],[2,1,3,4,5],[3,2,1,4,5],[4,2,3,1,5],[4,2,3,5,1]]
print('MAP@5 =',mapk(actual, predicted, k=5))
print('MP@5 =',mpk(actual, predicted, k=5))

('MAP@5 =', 0.43)
('MP@5 =', 0.27999999999999997)


3. Normalized Discounted Cumulative Gain  (NDCG)

The dcg function is modified from https://github.com/ChenglongChen/tensorflow-LTR/blob/master/src/metrics.py
According to wiki https://en.wikipedia.org/wiki/Discounted_cumulative_gain, 
Discounted cumulative gain (DCG) is a measure of ranking quality. In information retrieval, it is often used to measure effectiveness of web search engine algorithms or related applications.

The NDCG is calculated as follows
NDCG=DCG(predicted order)/DCG(sorted predicted order)
DCG(order)=sum(DCGi) i in order
DCGi=orderi/(ln2(i+1))

In [74]:
def dcg(predicted_order):
    """ Parameters: list predicted order
        Output: DCG
    """
    i = np.log2(1. + np.arange(1,len(predicted_order)+1))
    l = np.array(predicted_order)
    return np.sum(l/i)


def ndcg(score, top_ten=True):
    """ Parameters: list predicted order
        Output: nDCG=DCG(predicted order)/DCG(sorted predicted order)
    """
    end = 10 if top_ten else len(score)
    sorted_score = np.sort(score)[::-1]
    dcg_ = dcg(score[:end])
    if dcg_ == 0:
        return 0
    dcg_max = dcg(sorted_score[:end])
    return dcg_/dcg_max

In [75]:
predicted_order_ =[3,2,3,0,1,2,3,2]
print(dcg(predicted_order_))
print((ndcg(predicted_order_)))

8.492056442164959
0.935908621453514
